In [1]:
#!pip install simpletransformers

In [2]:
import torch
torch.__version__

/opt/anaconda3/envs/bert/bin/python


In [3]:
torch.cuda.empty_cache()
import sys
sys.  version
print('is there cuda available:',torch.cuda.is_available())
torch.cuda.device(0)
print('working of gpu',torch.cuda.current_device())
print('Name of gpu original or scientific',torch.cuda.get_device_name(0))


#want to change the gpu
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

'1.4.0'

In [8]:
#Basic libraries
import pandas as pd
import numpy as np

In [9]:
# install pytorch-pretrained-bert from PyPI
#!pip install pytorch-pretrained-bert

In [10]:
#All important libraries to run
import os
import math
import random
import csv
import sys

from sklearn import metrics
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.metrics import classification_report
import statistics as stats

#Cloned the bert_sklearn API here, only one time run
!git clone -b master https://github.com/charles9n/bert-sklearn
!cd bert-sklearn; pip install .

#Import the pretrained model
sys.path.append("../") 
from bert_sklearn import BertTokenClassifier, BertClassifier, BertRegressor
#It is use to load the trained model
from bert_sklearn import load_model

In [11]:
#Make our data into a comfortable form
def make_ner_data(filename, idx = 3):
    lines =  open(filename, encoding="utf8").read().strip()
    
    # find sentence-like boundaries
    lines = lines.split("\n\n")
    
    # split on newlines
    liness = []
    for line in lines:
        a = 0
        for i in range(len(line.split('\n'))):
            if line.split('\n')[i].split()[0] == '__':
                liness.append(line.split('\n')[a:i])
                a = i+1
                
    #tokens formation
    tokens = []
    for line in liness:
        token = []
        tokens.append(token)
        for l in line:
            #print(l.split()[0])
            token.append(l.split()[0])
            
    #labels formation
    labels = []
    for line in liness:
        label = []
        labels.append(label)
        for l in line:
            #print(l.split()[3])
            label.append(l.split()[3])
            
    #convert to df
    data= {'tokens': tokens, 'labels': labels}
    df=pd.DataFrame(data=data)
    return df

In [13]:
#Loaded the all text files
trainfile_cat="/home/user3/Documents/avan_phd/Objective_nlp/data_file/df_annotated_ner_cat.txt"
trainfile_pro="/home/user3/Documents/avan_phd/Objective_nlp/data_file/df_annotated_ner_pro.txt"
trainfile_cat_pro="/home/user3/Documents/avan_phd/Objective_nlp/data_file/df_annotated_ner_cat_pro.txt"

#Make data frame of each text file
df_cat = make_ner_data(trainfile_cat, idx=3)
df_cat_pro = make_ner_data(trainfile_cat_pro, idx=3) 
df_pro = make_ner_data(trainfile_pro, idx=3)

#concatenating the data frame along rows
final_ner_data = pd.concat([df_cat, df_cat_pro, df_pro], axis=0)

In [15]:
#The X and Y are decided 
X = final_ner_data.tokens
Y = final_ner_data.labels

In [37]:
#Split the data into train and test
from sklearn.model_selection import train_test_split
X_train,X_,y_train,y_=train_test_split(X,Y,test_size=0.2, random_state= 3)

In [38]:
#Total data points describe
print('shape of actual data', final_ner_data.shape[0])
print('shape of train data', X_train.shape[0])
print('shape of test data', X_test.shape[0])

shape of actual data 15247
shape of train data 12197
shape of test data 1525
shape of valid data 1525


In [39]:
#flatten the layer
def flatten(l):
    return [item for sublist in l for item in sublist]

In [40]:
#Tell uniqueness of token labels
import numpy as np
label_list = np.unique(flatten(Y))  # list of counts of uniques
print('Uniqueness of label:',label_list.values)

array(['0', 'catalyst', 'process'], dtype='<U8')

In [21]:
#Load the pretrained model of BertTokenClassifier
%%time
model = BertTokenClassifier(bert_model='scibert-basevocab-cased',
                            max_seq_length=178,
                            epochs=10,
                            gradient_accumulation_steps=2,
                            learning_rate=3e-5,
                            train_batch_size=32,
                            eval_batch_size = 32,
                            validation_fraction=0.1, 
                            ignore_label=['0'])

print(model)

Building sklearn token classifier...
BertTokenClassifier(bert_model='scibert-basevocab-cased', epochs=10,
                    eval_batch_size=32, gradient_accumulation_steps=2,
                    ignore_label=['0'], learning_rate=3e-05,
                    max_seq_length=178)
CPU times: user 2.81 ms, sys: 8 µs, total: 2.82 ms
Wall time: 2.14 ms


In [23]:
# finetune model on train data
model.fit(X_train, y_train)

Loading scibert-basevocab-cased model...
Defaulting to linear classifier/regressor
Loading Pytorch checkpoint
train data size: 10978, validation data size: 1219


Validating: 100%|██████████████████████████████████████████████████████| 39/39 [00:07<00:00,  5.38it/s]

Epoch 1, Train loss: 0.0189, Val loss: 0.0018, Val accy: 99.67%, f1: 96.27



Validating: 100%|██████████████████████████████████████████████████████| 39/39 [00:07<00:00,  5.36it/s]

Epoch 2, Train loss: 0.0013, Val loss: 0.0008, Val accy: 99.86%, f1: 98.42


...


Validating: 100%|██████████████████████████████████████████████████████| 39/39 [00:07<00:00,  5.36it/s]

Epoch 9, Train loss: 0.0002, Val loss: 0.0008, Val accy: 99.88%, f1: 98.57



Validating: 100%|██████████████████████████████████████████████████████| 39/39 [00:07<00:00,  5.37it/s]

Epoch 10, Train loss: 0.0002, Val loss: 0.0008, Val accy: 99.88%, f1: 98.64


BertTokenClassifier(bert_model='scibert-basevocab-cased', do_lower_case=False,
                    epochs=10, eval_batch_size=32,
                    gradient_accumulation_steps=2, ignore_label=['0'],
                    label_list=array(['0', 'catalyst', 'process'], dtype='<U8'),
                    learning_rate=3e-05, max_seq_length=178)

In [24]:
#save model to disk
savefile = 'ner_model10_178.bin'
model.save(savefile)

In [22]:
#load the save the model
savefile = 'ner_model10_178.bin'
# load model from disk
model = load_model(savefile)

Loading model from ner_model10_178.bin...
Defaulting to linear classifier/regressor
Building sklearn token classifier...


In [46]:
# get predictions on test data
y_preds = model.predict(X_test)

# print report on classifier stats
print(classification_report(flatten(y_test), flatten(y_preds)))

#Accuracy of test
print('Accuracy of test:', accuracy_score(flatten(y_test), flatten(y_preds)))

Predicting: 100%|██████████████████████████████████████████████████████| 48/48 [00:08<00:00,  5.59it/s]


              precision    recall  f1-score   support

           0       1.00      1.00      1.00     57294
    catalyst       0.95      0.92      0.93       413
     process       0.99      0.97      0.98      2083

    accuracy                           1.00     59790
   macro avg       0.98      0.96      0.97     59790
weighted avg       1.00      1.00      1.00     59790

Accuracy of test: 0.9975581200869711


In [49]:
#Print the confusion matrix
metrics.confusion_matrix(flatten(y_test), flatten(y_preds))

array([[57248,    20,    26],
       [   34,   379,     0],
       [   66,     0,  2017]])

In [47]:
# get predictions on test data
y_pred_train = model.predict(X_train)

# print report on classifier stats
print(classification_report(flatten(y_train), flatten(y_pred_train)))

#Accuracy of train
print('Accuracy of train:', accuracy_score(flatten(y_train), flatten(y_pred_train)))

Predicting: 100%|████████████████████████████████████████████████████| 382/382 [01:06<00:00,  5.71it/s]


              precision    recall  f1-score   support

           0       1.00      1.00      1.00    461242
    catalyst       0.98      0.98      0.98      3466
     process       0.99      0.97      0.98     17142

    accuracy                           1.00    481850
   macro avg       0.99      0.98      0.99    481850
weighted avg       1.00      1.00      1.00    481850

Accuracy of test: 0.9984601016913978


In [50]:
#Print the confusion matrix
metrics.confusion_matrix(flatten(y_train), flatten(y_pred_train))

array([[461085,     69,     88],
       [    78,   3388,      0],
       [   507,      0,  16635]])

In [23]:
#predict the tags to each token in sentence
from nltk.tokenize import word_tokenize 
import sys
from termcolor import colored, cprint
 
def color_text_(tag_predict, text):
    text_list  = []

    a = word_tokenize(text)
    for i in range(len(a)):

        if  tag_predicts[i] == 'catalyst':
            
          #print(i, colored(text.split()[i], 'white', 'on_red'))
          #print(colored(text.split()[i], 'white', 'on_red'))
          #a = colored(text.split()[i], 'white', 'on_red')
            text_list.append(colored(a[i], 'white', 'on_red'))

        elif tag_predicts[i] == 'process':
            text_list.append(colored(a[i], 'white', 'on_green'))
        else:
            text_list.append(a[i])

  #print(" ".join(text_list))
    return " ".join(text_list)

In [62]:
#predict the tags to each token in sentence
#text = 'This Cu is used as catalyst at  100 kelvin and 10 bar pressure'
#text1 = 'A frontal chromatographic version of this technique has been developed which is very suitable for in situ measurements anammed rper min flow rate ) at different temperatures ( 400 , 500 , and 650 celcius ) with a heating rate of 10 celcius per min'
text1 = 'Several studies have reported the effective accomplishment of these reactions type in the presence of Pd alloys catalysts in relative high pressure'
tag_predicts  = model.tag_text(text1) 
print(color_text_(tag_predicts, text1))

Predicting: 100%|████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.37it/s]

             token predicted tags
0          Several              0
1          studies              0
2             have              0
3         reported              0
4              the              0
5        effective              0
6   accomplishment              0
7               of              0
8            these              0
9        reactions              0
10            type              0
11              in              0
12             the              0
13        presence              0
14              of              0
15              Pd       catalyst
16          alloys              0
17       catalysts              0
18              in              0
19        relative              0
20            high              0
21        pressure        process
Several studies have reported the effective accomplishment of these reactions type in the presence of Pd alloys catalysts in relative high pressure
